<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/guides/agentic-rag/agentic_rag_pt3_sequential.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Performing Tasks Sequentially

Compare two user queries to a RAG chatbot, "What was Apple's revenue in 2023?" and "What was the revenue in of the most valuable company in the US in 2023?".

While the first query is straightforward to handle, the second query requires breaking down into two steps:
1. Identify the most valuable company in the US in 2023
2. Get the revenue of the company in 2023

These steps need to happen in a sequence rather than all at once. This is because the information retrieved from the first step is required to inform the second step.

This is an example of sequential reasoning. In this tutorial, you will learn how agentic RAG with Cohere handles sequential reasoning, and in particular:
- Multi-step tool calling
- Multi-step, parallel tool calling
- Self-correction

You'll learn these by building an agent that answers questions about using Cohere.

## Setup

To get started, first we need to install the `cohere` library and create a Cohere client.

In [1]:
import json
import os
import cohere

from tool_def import (
    search_developer_docs,
    search_developer_docs_tool,
    search_internet,
    search_internet_tool,
    search_code_examples,
    search_code_examples_tool,
)

co = cohere.ClientV2(api_key=os.environ["COHERE_API_KEY"])

## Setting up the tools

We set up the same set of tools as in Part 1. If you want further details on how to set up the tools, check out Part 1.

In [2]:
functions_map = {
    "search_developer_docs": search_developer_docs,
    "search_internet": search_internet,
    "search_code_examples": search_code_examples,
}

## Running an agentic RAG workflow

We create a `run_agent` function to run the agentic RAG workflow, the same as in Part 1. If you want further details on how to set up the tools, check out Part 1.

In [3]:
tools = [
    search_developer_docs_tool,
    search_internet_tool,
    search_code_examples_tool
]

In [4]:
system_message="""## Task and Context
You are an assistant who helps developers use Cohere. You are equipped with a number of tools that can provide different types of information. If you can't find the information you need from one tool, you should try other tools if there is a possibility that they could provide the information you need."""

In [5]:
model = "command-r-plus-08-2024"

def run_agent(query, messages=None):
    if messages is None:
        messages = []
        
    if "system" not in {m.get("role") for m in messages}:
        messages.append({"role": "system", "content": system_message})
    
    # Step 1: get user message
    print(f"QUESTION:\n{query}")
    print("="*50)
    
    messages.append({"role": "user", "content": query})

    # Step 2: Generate tool calls (if any)
    response = co.chat(
        model=model,
        messages=messages,
        tools=tools,
        temperature=0.1
    )

    while response.message.tool_calls:
        
        print("TOOL PLAN:")
        print(response.message.tool_plan,"\n")
        print("TOOL CALLS:")
        for tc in response.message.tool_calls:
            print(f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}")
        print("="*50)

        messages.append({"role": "assistant", "tool_calls": response.message.tool_calls, "tool_plan": response.message.tool_plan})        
        
        # Step 3: Get tool results
        for tc in response.message.tool_calls:
            tool_result = functions_map[tc.function.name](**json.loads(tc.function.arguments))
            tool_content = []
            for data in tool_result:
                tool_content.append({"type": "document", "document": {"data": json.dumps(data)}})
                # Optional: add an "id" field in the "document" object, otherwise IDs are auto-generated
            messages.append({"role": "tool", "tool_call_id": tc.id, "content": tool_content})
        
        # Step 4: Generate response and citations 
        response = co.chat(
            model=model,
            messages=messages,
            tools=tools,
            temperature=0.1
        )
    
    messages.append({"role": "assistant", "content": response.message.content[0].text})
        
    # Print final response
    print("RESPONSE:")
    print(response.message.content[0].text)
    print("="*50)
    
    # Print citations (if any)
    verbose_source = False # Change to True to display the contents of a source
    if response.message.citations:
        print("CITATIONS:\n")
        for citation in response.message.citations:
            print(f"Start: {citation.start}| End:{citation.end}| Text:'{citation.text}' ")
            print("Sources:")
            for idx, source in enumerate(citation.sources):
                print(f"{idx+1}. {source.id}")
                if verbose_source:
                    print(f"{source.tool_output}")
            print("\n")
                    
    return messages

## Multi-step tool calling

Let's ask the agent a few questions, starting with this one about a specific feature. The user is asking about two things, first about a features to reorder search results, and second about code examples for that feature.

In this case, the agent first needs to identify what that feature is before it can answer the second part of the question.

This is reflected in the agent's tool plan, which describes the steps it will take to answer the question.

So, it first calls the `search_developer_docs` tool to find the feature. 

It then finds out that the feature is Rerank. And using this information, it calls the `search_code_examples` tool to find code examples for that feature.

Finally, it uses the retrieved information to answer both parts of the user's question.

In [6]:
messages = run_agent("What's the Cohere feature to reorder search results? Do you have any code examples on that?")

QUESTION:
What's the Cohere feature to reorder search results? Do you have any code examples on that?
TOOL PLAN:
I will search for the Cohere feature to reorder search results. Then I will search for code examples on that. 

TOOL CALLS:
Tool name: search_developer_docs | Parameters: {"query":"reorder search results"}
TOOL PLAN:
I found that the Cohere feature to reorder search results is called the Rerank endpoint. I will now search for code examples on this. 

TOOL CALLS:
Tool name: search_code_examples | Parameters: {"query":"Rerank endpoint"}
RESPONSE:
The Cohere feature to reorder search results is the Rerank endpoint. This endpoint takes in a query and a list of texts and produces an ordered array with each text assigned a relevance score.

Here is a code example that uses the Rerank endpoint:

RAG With Chat Embed and Rerank via Pinecone
CITATIONS:

Start: 52| End:68| Text:'Rerank endpoint.' 
Sources:
1. search_developer_docs_07rw24b2sa29:0


Start: 83| End:192| Text:'takes in a q

## Multi-step, parallel tool calling

We saw in Part 2 how the Cohere API suports tool calling to happen in parallel, and now in a sequence. That also means that both scenarios can happen at the same time.

Here's an examples. Suppose we ask the agent to find the leaders of the top 3 countries with the largest oil reserves.

In the first step, it looks up the internet for information about the 3 countries with the largest oil reserves.

And in the second step, it performs parallel searches for the leaders of the 3 identified countries.

In [7]:
messages = run_agent("Who are the leaders of the top 3 countries with the largest oil reserves.")

QUESTION:
Who are the leaders of the top 3 countries with the largest oil reserves.
TOOL PLAN:
I will search for the top 3 countries with the largest oil reserves. Then I will search for the leaders of each of these countries. 

TOOL CALLS:
Tool name: search_internet | Parameters: {"query":"top 3 countries with the largest oil reserves"}
TOOL PLAN:
I have found that the top three countries with the largest oil reserves are Venezuela, Saudi Arabia and Canada. Now I need to find out who the leaders of these countries are. 

TOOL CALLS:
Tool name: search_internet | Parameters: {"query":"Who is the leader of Venezuela?"}
Tool name: search_internet | Parameters: {"query":"Who is the leader of Saudi Arabia?"}
Tool name: search_internet | Parameters: {"query":"Who is the leader of Canada?"}
RESPONSE:
The top three countries with the largest oil reserves are Venezuela, Saudi Arabia, and Canada.

The leader of Venezuela is Nicolás Maduro. Maduro was born on November 23, 1962, in Caracas, Venezu

## Self-correction

The concept of sequential reasoning is useful in a broader sense, and in particular, where the agent needs to adapt and change its plan midway in a task. 

In other words, it allows the agent to self-correct.

To illustrate this, let's look at an example. Here, the user is asking about the Cohere safety mode feature.

Given the nature of the question, the agent correctly identifies that it needs to find required information via the `search_developer_docs` tool.

However, we know that the tool doesn't contain this information because we have only added a small sample of documents there.

As a result, the agent, having received the documents back without any relevant information, decides to search the internet instead. This is also helped by the fact the we have added specific instructions in the `search_internet` tool to search the internet for information not found in the developer documentation.

It finally has the information it needs, and uses it to answer the user's question.

This highlights another important aspect of agentic RAG where it allows a RAG system to be flexible. This is achieved by powering the retrieval component with an LLM.

On the other hand, a standard RAG system would typically hand-engineer this, and hence, is more rigid.

In [8]:
messages = run_agent("How does the Cohere safety mode feature work.")

QUESTION:
How does the Cohere safety mode feature work.
TOOL PLAN:
I will search for 'How does the Cohere safety mode feature work?' 

TOOL CALLS:
Tool name: search_developer_docs | Parameters: {"query":"How does the Cohere safety mode feature work?"}
TOOL PLAN:
I could not find any information about the Cohere safety mode feature in the developer documentation. I will now search the internet to see if I can find any information. 

TOOL CALLS:
Tool name: search_internet | Parameters: {"query":"How does the Cohere safety mode feature work?"}
RESPONSE:
Cohere's Safety Modes aim to illustrate what model behaviours will look like under specific scenarios, thereby introducing a nuanced approach that is sensitive to context. By transparently communicating the strengths and boundaries of each mode, Cohere intends to set clear usage expectations while keeping safety as its top priority.

Safety Modes work with Cohere's newest refreshed models, but not with older iterations. Users can switch be

## Summary

In this tutorial, you learned about:
- How multi-step tool calling works
- How multi-step, parallel tool calling works
- How multi-step tool calling enables an agent to self-correct, and hence, be more flexible

However, up till now, we have only worked with purely unstructured data, the type of data we typicallyencounter in a standard RAG system.

In the coming chapters, we'll add another complexity to the agentic RAG system, which is working with semi-structured and structured data. This adds another dimension to the agent's flexibility, which is dealing with a more diverse set of data sources.

In Part 4, you will learn how to work with semi-structured data.